In [5]:
import numpy as np
from scipy import stats as st
from random import shuffle

In [49]:
#initialize inputs
twain = [.25,.262,.217,.24,.23,.229,.235,.217]
snodgrass = [.209,.205,.196,.21,.202,.207,.224,.223,.22,.201]
#get the mean of each list
tmu = np.mean(twain)
smu=np.mean(snodgrass)
#get delta
delta = tmu - smu
#compute the variance of each list
vTwain = np.sum((twain - tmu)**2) / len(twain)
vSnod = np.sum((snodgrass - smu)**2) / len(snodgrass)
#compute standard error
se = np.sqrt(vTwain/len(twain) + vSnod/len(snodgrass))
#compute Wald thingy
W = delta/se
#compute the p_value and print results
p_value = 2*st.norm.cdf(-abs(W))
print("p_value:",p_value)
#compute confidence interval & print results
z=st.norm.ppf(.975)
C=(delta-se*z,delta+se*z)
print("Confidence interval:",C)

p_value: 1.7848711894915704e-05
Confidence interval: (0.013741861224727436, 0.03685813877527254)


We reject the null hypothesis.

In [50]:
#Stack the 2 lists
stacked = twain + snodgrass
numReject = 0
numTrials = 100000
#loop through a bunch of permutations of this stacked list to approximate 
#the permutation method. (since 18! is too much to compute)
for i in range(numTrials):
    shuffle(stacked)
    T = np.abs(np.mean(stacked[:8]) - np.mean(stacked[8:]))
    if T > delta:
        numReject +=1
        
#compute p_value & print results        
p_value = numReject / numTrials
print("p_value:",p_value)

p_value: 0.00054


We reject the null hypothesis

In [45]:
#initialize inputs
p_hats = [45/80,26/75,52/85,35/67,37/85]
patients = [80,75,85,67,85]
p0 = p_hats[0]
#get the z for 95% confidence
z = st.norm.ppf(.975)
rejects = []
p_vals = []
#loop through the p_hats testing it with the Wald test & print 
#which drugs we will reject the null hypothesis for
for i in range(1,5):
    p_hat = p_hats[i]
    delta = p_hat - p0
    se = np.sqrt(p_hat*(1-p_hat)/patients[i] + p0*(1-p0)/80)
    W = delta / se
    p_vals.append(2*st.norm.cdf(-abs(W)))
    if np.abs(W) > z:
        rejects.append(i)
print(rejects)

[1]


We accept the null hypothesis for all except the drug Chlorpromazine

In [47]:
#Bonferroni method
a = 0.05 / 4
z = st.norm.ppf(1-a/2)
rejects = []
#same thing as a, but for the bonferroni method
for i in range(1,5):
    p_hat = p_hats[i]
    delta = p_hat - p0
    se = np.sqrt(p_hat*(1-p_hat)/patients[i] + p0*(1-p0)/80)
    W = delta / se
    if np.abs(W) > z:
        rejects.append(i)
print(rejects)

[1]


We accept the null hypothesis for all except the drug Chlorpromazine

In [51]:
#FDR Method
Cm,m = 1,4
a = 0.05
l = [i*a/(Cm*m) for i in range(1,5)]
spval = np.sort(p_vals)
#get the largest pvalue where the sorted list is less than l
T = np.max(spval[spval < np.array(l)])
#reject null hypothesis if p_value is <= T
print(p_vals <= T)

[ True False False False]


We accept the null hypothesis for all except the drug Chlorpromazine